<a href="https://colab.research.google.com/github/ajinkyabhabad/Algorithmic-Digital-Marketing/blob/master/Olist%20E-Commerce%20Recommendation%20System/3.Recommendation/ALS_Regular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Alternating Least Squares for regular customers

ALS (Alternating Least Squares) is an implicit recommendation algorithm to make a recommendation of products to the users. It uses correlation matrix based of product rating provided by customer to recommend the products to customers.

In [ ]:
import pandas as pd
import os
import numpy as np 
import random

import random
import pandas as pd
import numpy as np
import pickle
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Regular.csv to Regular (1).csv


In [ ]:
import io
Olist_Reg_db = pd.read_csv(io.BytesIO(uploaded['Regular.csv']))

In [ ]:
Olist_Reg_db

,PRODUCT_ID,ORDER_ID,CUSTOMER_ID,PRODUCT_CATEGORY_NAME,PAYMENT_VALUE,ORDER_DELIVERED_CUSTOMER_DATE,CUSTOMER_UNIQUE_ID,REVIEW_SCORE,CUSTOMER_CITY,RECENCY,FREQUENCY,MONETARY,RANK,R_Quartile,F_Quartile,M_Quartile,RFMScore,Type
0,1eba879220bd0981a0e2fbab499ed4e1,62073ec6b54b8e6322037fc0f3591ad3,368f207cdc8f5c93a2b2a5d4cce96ba8,moveis_escritorio,1316.14,2018-04-24 20:36:50,f9172a6495d46451776be8bc8e46032d,2,japeri,175.0,2,2632.28,120.0,3,4,4,344,Regular
1,680874c570dad71c0a2844cfbf417054,18f4178d9b3c33fa79c1a0d5eb214023,1f68e24da06f36cb8a5243e23c390bcf,moveis_decoracao,305.89,2018-05-14 16:48:43,c01ac50f437f46cc8fffb6d1a01d8631,1,belo horizonte,155.0,2,611.78,3584.0,3,4,4,344,Regular
2,680874c570dad71c0a2844cfbf417054,d2ebecd13598667c0e664cf7c3eace01,f59d1a0dc2219d4bdb4fc23f0d1d3c07,moveis_decoracao,109.71,2018-02-24 14:48:54,dbe1f72423b35ecd39ebd4170bed945c,1,rio de janeiro,234.0,2,219.42,17636.0,3,4,4,344,Regular
3,bbf920aa6ac72007a726e6f419156f8f,1d23106803c48c391366ff224513fb7f,b2e6ad00199e893fdee50615ccfda0fd,moveis_decoracao,214.19,2018-02-09 21:38:39,76b61cd27ca2933a2bd2669762fb229a,1,pelotas,249.0,4,856.76,1857.0,3,4,4,344,Regular
4,4c8056caed5757810b9a68b83c3759e6,d7dd7b5a139c49fb68a6977797599973,545c79b40c22b0a53abea88e0948c862,malas_acessorios,377.94,2018-04-16 17:51:09,5d8f5f50fd2c7fbe8b335d945105a37f,4,sao paulo,183.0,2,755.88,2396.0,3,4,4,344,Regular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1723,a0b7d5a992ccda646f2d34e418fff5a0,201797d3f567dc70cc5732ecf055e346,bbc8c12f7afeba03337723117f0a6f3e,moveis_decoracao,9.92,2018-03-27 16:42:16,254f01c0c4853e5c23a801f29069f932,5,santana de parnaiba,203.0,4,323.72,9837.0,3,4,4,344,Regular
1724,a0b7d5a992ccda646f2d34e418fff5a0,201797d3f567dc70cc5732ecf055e346,bbc8c12f7afeba03337723117f0a6f3e,moveis_decoracao,151.94,2018-03-27 16:42:16,254f01c0c4853e5c23a801f29069f932,5,santana de parnaiba,203.0,4,323.72,9837.0,3,4,4,344,Regular
1725,a0b7d5a992ccda646f2d34e418fff5a0,531a74ff9a6617b0cae88c4b67646988,177a576c3346d73fa4b179ee92d61dbd,moveis_decoracao,178.61,2018-02-19 15:33:49,0b4c8216356414f6b079d5cb85bad4aa,5,uberlandia,239.0,2,357.22,8385.0,3,4,4,344,Regular
1726,a0b7d5a992ccda646f2d34e418fff5a0,be8825818ef90fa22e25614dad11854d,ca3f383440e3514f41b0ebdb9b7b88f2,moveis_decoracao,223.00,2018-04-18 13:58:42,396adb4d8570f382a271ba14a73d0190,5,sao paulo,181.0,2,446.00,5806.5,3,4,4,344,Regular


In [ ]:
features = ['CUSTOMER_ID', 'PRODUCT_ID', 'REVIEW_SCORE']
Olist_Reg_db_ = Olist_Reg_db[features]
Olist_Reg_db_ = Olist_Reg_db_.rename(columns={'Review Score': 'Ratings'})
Olist_Reg_db_ = Olist_Reg_db_.drop_duplicates()

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
Olist_Reg_db_.CUSTOMER_ID.unique

<bound method Series.unique of 0       368f207cdc8f5c93a2b2a5d4cce96ba8
1       1f68e24da06f36cb8a5243e23c390bcf
2       f59d1a0dc2219d4bdb4fc23f0d1d3c07
3       b2e6ad00199e893fdee50615ccfda0fd
4       545c79b40c22b0a53abea88e0948c862
                      ...               
1722    2a6179cf815c302b5b94a2bab0af39d4
1723    bbc8c12f7afeba03337723117f0a6f3e
1725    177a576c3346d73fa4b179ee92d61dbd
1726    ca3f383440e3514f41b0ebdb9b7b88f2
1727    2f1a3baba1b833c18c5da7b721b360b2
Name: CUSTOMER_ID, Length: 1537, dtype: object>

In [ ]:
# pivot ratings into movie features
df_product_features = Olist_Reg_db_.reset_index().pivot_table(
    index='CUSTOMER_ID',
    columns='PRODUCT_ID',
    values='REVIEW_SCORE'
).fillna(0)
# convert dataframe of movie features to scipy sparse matrix
mat_product_features = csr_matrix(df_product_features.values)

In [ ]:
df_product_features.head()

PRODUCT_ID,00ae7076313576f94d9107599d79a978,00faa46f36261af8bbf3a4d37fa4841b,0105b5323d24fc655f73052694dbbb3a,01365901d8f2247976a7e5d4c98c52c0,013ee64977aaa6b2b25475095162e0e9,01422266d7a3131403364787ef9dab11,014a8a503291921f7b004a5215bb3c36,0160785b697a5ea87d52949bd097ce05,017692475c1c954ff597feda05131d73,01ff1ff8aa5dec93e9938b989393a4ca,02357c69f595e004b50ba115b3f88d63,02475368dfb38934fe55f574024fe1d7,02ed3b595447e948efc3eac897e4df0e,045e41f597e72ebfe5e9de50788fbf03,049ac5dce8536884a652ce73c58022ab,04f982a576e5d149919df37e292fa79e,056d012d264624accb7f73d31caee034,0573b564a90d2d9f9c460eb963da8c33,058b372f969b59e8c4a27e224243909c,05c11568c221bcac5790a8720019d141,060ea530ba6184d49f7b1cc031c4c97e,063d96e7b9190a8da0b8920906578383,0654d3c1e3efc7e7cd9dbde9cb743df2,06c950c36d589b39b34f5dc1d4a75e29,06dce482ace017e78349b6f1fa83a022,06edb72f1e0c64b14c5b79353f7abea3,06f85533c1d65036e055ed561d0b1a2e,071b5dd29feece041c356d172474b113,073afdaeb2d71d4da3ce0c6935614289,078e8db43688ea060cf754e7bd2b92e7,0797e47ffb3917b66fa62245a283bd9e,07ffc018eaf23e086370dea42c74077b,084b2ab1a0a79a9757008062ddf8b042,086351823300e0339f6955b27998c186,08c816d9a716567bf436173ef6f5f12e,094d192d5b25e0bae4c3e2f970a53439,095fc85516c1210e6dbed95ed8041ee0,0981ba54d8ddc68299f7c7a7d23bb2c9,098c663630a7a81f7d7f42a2ce0b02d7,09c3d39641970009c198caed304ccfc4,...,f77f34a4f2b8f31e2bcc50a478888064,f7c3fc384cd3ac71650982c2c6fcbe77,f7d5edd55cc83003c9ff58de184833d5,f819f0c84a64f02d3a5606ca95edd272,f83a07c7e2a2609ea4c66d3d880b2a66,f86ab7cb275f6f3343848966310160e3,f89cd865cac300a9bf1320dd8f0fa223,f8b624d4e475bb8d1bddf1b65c6a64f6,f95d9369ebf76f811194c72bd22ff244,f97a136cf6e5f5d9333b513afc8b153e,f99e92b9472a8d08d27cfc7e98292b42,fa304c5424fd292aed9c35eb36d9f495,fa559a64fdb8719b675a8baf140b3d9e,fb01a5fc09b9b9563c2ee41a22f07d54,fb2f2ec90b4ee90ad257bbf89d01247e,fb55982be901439613a95940feefd9ee,fb6782985a98aa8a59238f58239f6f1e,fb92dc4602bd9224e0e267496748afc5,fbc1488c1a1e72ba175f53ab29a248e8,fbcd9b07bda0301e7e776fc1ab26fdfe,fc1d8637c0268af3db482c14b7ef8e75,fc276546b539ea9ad62827c2a5e2ae42,fc2d17e60bdd3349a25f3ca6a802a91e,fc5a20562c99bbc5229b175d0a88ea7a,fc5bf9e50dbfb45e7634bd8c045411c5,fc5dd987f12a7b823a76a44aa1ba88f6,fc6d90a253003d53136e1b444beb55dc,fc831da5307b1a3e23ed0259a2b6b2a1,fcf28afb1353f2f12ea041dd74954226,fcf6ad274391aea29f5d6e5ef9da5050,fd5c5a67cd369732c2ac20dbd574d1d4,fd925ce72399e23cbc5d75e224fc2b57,fdbfd4cec6db6e89f22d9aa2c4839eed,fe4dece0c663ba4f1f9f2d04d3b93a97,fe61253e50ad6ee9f1d180da22ce5eef,fe6f03cedd96bcbc5c8726c5963fd35c,fe9dfbe7f974621789683b7b78be2a16,feb4ade62e32b8d74c6f69f635057964,fec2d939a171210847d8f2d102f0dba5,fff1059cd247279f3726b7696c66e44e
CUSTOMER_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
006431d77c665533ac99b501d8a6ab06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
007da0ded5420c2ac9cdcdc758841cde,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0096dd625383caa8afe2c028018bf4ce,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00c2781c163b9f057924ed8c45c724d4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [ ]:
df_product_features.shape

(776, 1193)

In [ ]:
X = df_product_features.T
X.head()

CUSTOMER_ID,006431d77c665533ac99b501d8a6ab06,007da0ded5420c2ac9cdcdc758841cde,0096dd625383caa8afe2c028018bf4ce,00c2781c163b9f057924ed8c45c724d4,01414201d70bd508e1eee33ff67d2453,01503d0b64036165c2f202b538de9308,0195699b3388a65b55eab3dd04f48d1a,01cc8550598fb6140923843e6c06326e,0222b08d0816bba0acbcc31ba1d6cc1a,022bdd440f8a37e8ba62d67287135a75,023fafd700dff9fcc64d28e2defe98b9,02af4338e00cfed3af1a15854865879c,03346340b055f4770567d2a28ca52d4e,039a50ec3dc563eaf0bb3ecf176cdf99,03e3a430d50d6e1123627d133c65d5c4,03e454af33e93cda33e1b22175ab8361,03e5e57dd377a3da84cec48a7bbfba2a,03f2dda26a1b89b7cf91dcd8a41feb96,040a6b33a803ef269d0bf0c285cb2390,042b53f9e609289d8f6fcbd59b4acf63,04612c0745e93a575dac8d71974c805d,0499e8d702425ef8b10da2172f9fec96,053a441e831f4858b13182fe2671e967,05486eac340ae6a8f4837c6ab027d5ed,054c3d6634ff610c71c882d6f157bb34,0641f390b82e408dd226cd5e98327258,06d99479baac3413561b21bd7ac27767,06deea7d56068efd37c65f8daff0e03b,078d1ab1e800b24415a6d90e582df853,08029b648bf207edf7c24f3673382bf2,0879c47e542184edf3c921d2fee17084,08a0a61f98f22952682f7e463ef2d43e,08e7487578bd3fb98d11c5d95f1e5db5,0900a5f39daf5f37e39f42894f36a85d,0980d72e94bede8b0f7d8a4cd32511c8,0987ae71e7ad93d01d1202b7299fbb50,098ff9996de8b54d861bf8da445ba2ca,09ca1779c44a99be8f22fb0acfd3eefd,09cc193befa14d698869667e92148ab2,09e01f974b9533109a46281624e39808,...,f3ad475cc60e554f7113f18ab06810ca,f40e11fcab48bea5f847eff467a841a0,f40fb1a58c563f8e1581d3e93ce79507,f43d9a5994fb99bb2e14160e12666442,f4608a440b868f3f57f378c4b8cd643f,f4984d9f9ce9b3201db4932ea329cb40,f4c13379ddd0ed4f4fc1c0b49c2c5e51,f4e7ab7598cbef510a638a6068fbc56a,f59d1a0dc2219d4bdb4fc23f0d1d3c07,f665e06d0376b22d2ac11e29e369d6b1,f6a6b6537e91cdb94efd469d48d78690,f6a9bfc46722d91dc2306b08a0b8e11a,f6abfe23cd8425cc79033317e567b969,f74f7d4e2664b99e1cd78ef0f906f4cb,f7c7eee947743e900cad930670b94341,f7fd335725f778c5e85be886b3ac21ee,f85444670597b08b2c8a44beda3d72db,f8a069c2a2ce5ac98d668ad6a7950cb8,f8d4fd17a8f45986298a522b397522d0,f99b5ad76ac1085ae3628a3dc0eff0af,f9b4364d72fa2633b5eaaf6626e19cd9,f9efa1e08ec36a3bd42cdeb68995af0b,fa8047426049005aaa90db11b5fd39ad,faa2debf21e388f071f302ef5dc5e1cc,fab4d0e05c214c4baa684e6107d77757,fac3296ec4f116faccfa6a6d7e18b42f,fac60d7c4df3896a5af5db6c91e03797,fb54e4c6db413239a97873a13c366310,fbd2a08da7b03d6aa249b3b68ce13170,fbdea18ec346682a0dd7c542c56e74ff,fcfa6b64765235e944c28d83f99ee9dd,fd03d461d47cf24785f5e7ee7a1c9708,fd2ae8c714a773f85ef1b53fbebeda1f,fd2bc7895fa1c769be64f398f45c8dcf,fd44d124e2a3efe1398e5b4919b17413,fd92e79c1dfcd1ddae4d4630fef5bb7e,fec7723925e22866dab2debf6a38b695,fed5b7561b589c773a4a7d7028d0ae9b,ff349ea42c06633992fc3c94c3da0f58,ffc436ef9df6ede268ee1548a0dfbbba
PRODUCT_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
00ae7076313576f94d9107599d79a978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00faa46f36261af8bbf3a4d37fa4841b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0105b5323d24fc655f73052694dbbb3a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01365901d8f2247976a7e5d4c98c52c0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [ ]:
X.index[:-20]

Index(['00ae7076313576f94d9107599d79a978', '00faa46f36261af8bbf3a4d37fa4841b',
       '0105b5323d24fc655f73052694dbbb3a', '01365901d8f2247976a7e5d4c98c52c0',
       '013ee64977aaa6b2b25475095162e0e9', '01422266d7a3131403364787ef9dab11',
       '014a8a503291921f7b004a5215bb3c36', '0160785b697a5ea87d52949bd097ce05',
       '017692475c1c954ff597feda05131d73', '01ff1ff8aa5dec93e9938b989393a4ca',
       ...
       'f99e92b9472a8d08d27cfc7e98292b42', 'fa304c5424fd292aed9c35eb36d9f495',
       'fa559a64fdb8719b675a8baf140b3d9e', 'fb01a5fc09b9b9563c2ee41a22f07d54',
       'fb2f2ec90b4ee90ad257bbf89d01247e', 'fb55982be901439613a95940feefd9ee',
       'fb6782985a98aa8a59238f58239f6f1e', 'fb92dc4602bd9224e0e267496748afc5',
       'fbc1488c1a1e72ba175f53ab29a248e8', 'fbcd9b07bda0301e7e776fc1ab26fdfe'],
      dtype='object', name='PRODUCT_ID', length=1173)

In [ ]:
SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

(1193, 10)

In [ ]:
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

(1193, 1193)

In [ ]:
correlation_matrix

array([[ 1.        ,  0.3375158 ,  0.13872829, ...,  0.74705912,
         0.32030689,  0.24635814],
       [ 0.3375158 ,  1.        , -0.47449314, ..., -0.29869214,
         0.52021476, -0.49346152],
       [ 0.13872829, -0.47449314,  1.        , ...,  0.47674136,
        -0.20576848,  0.18219044],
       ...,
       [ 0.74705912, -0.29869214,  0.47674136, ...,  1.        ,
        -0.02487883,  0.35444819],
       [ 0.32030689,  0.52021476, -0.20576848, ..., -0.02487883,
         1.        , -0.52338223],
       [ 0.24635814, -0.49346152,  0.18219044, ...,  0.35444819,
        -0.52338223,  1.        ]])

In [ ]:
fave_prod = Olist_Reg_db_.groupby(['CUSTOMER_ID']).max()['PRODUCT_ID'].to_frame()

In [ ]:
fave_prod = fave_prod.reset_index()
fave_prod

,CUSTOMER_ID,PRODUCT_ID
0,006431d77c665533ac99b501d8a6ab06,d13a4acdb64c202e774e815db547e108
1,007da0ded5420c2ac9cdcdc758841cde,cfa14c9ef22bd6c41e20df06de9dfa76
2,0096dd625383caa8afe2c028018bf4ce,13db47eae724e2848e12b71a617a3a41
3,00c2781c163b9f057924ed8c45c724d4,e83e5f5ff7ef69f7a09c848795df56a7
4,01414201d70bd508e1eee33ff67d2453,e8aa7bbd775831bc6bdc900a9bf5d75a
...,...,...
771,fd92e79c1dfcd1ddae4d4630fef5bb7e,a9bd39d11fe4d2d378c44eee9b11d9ce
772,fec7723925e22866dab2debf6a38b695,979473e8af1d4eaf4b889a0ba32834ce
773,fed5b7561b589c773a4a7d7028d0ae9b,70906e04da1eebf3d1b8791bd09ffe85
774,ff349ea42c06633992fc3c94c3da0f58,3f14d740544f37ece8a9e7bc8349797e


In [ ]:
def get_prod_id(customer_id):
    
    product_id = fave_prod[fave_prod.CUSTOMER_ID == customer_id]['PRODUCT_ID']
    return product_id
    
    
prd_id = get_prod_id('fed5b7561b589c773a4a7d7028d0ae9b')
Product_id = prd_id.iloc[0]
Product_id

'70906e04da1eebf3d1b8791bd09ffe85'

In [ ]:
product_names = list(X.index)
product_ID_Idx = product_names.index(Product_id)
product_ID_Idx

538

In [ ]:
correlation_product_ID = correlation_matrix[product_ID_Idx]
correlation_product_ID.shape

(1193,)

Recommending top 10 highly coorelated products

In [ ]:
Recommend = list(X.index[correlation_product_ID > 0.70])
# Removes the item already bought by the customer
Recommend.remove(Product_id) 
Recommend[0:20]

['01422266d7a3131403364787ef9dab11',
 '01ff1ff8aa5dec93e9938b989393a4ca',
 '045e41f597e72ebfe5e9de50788fbf03',
 '0573b564a90d2d9f9c460eb963da8c33',
 '058b372f969b59e8c4a27e224243909c',
 '05c11568c221bcac5790a8720019d141',
 '06dce482ace017e78349b6f1fa83a022',
 '07ffc018eaf23e086370dea42c74077b',
 '095fc85516c1210e6dbed95ed8041ee0',
 '09c3d39641970009c198caed304ccfc4',
 '0c9452f6a812935c0615faee8bf8646b',
 '0f4a3e29a1c122f94fe4c48c49d9cfa9',
 '0fa0bd7ef862a1a28d6cd9701fc97f6c',
 '10990aaa402c4692f9eb94cbc354e53c',
 '10aecaf5be4119fd6b8b9e2e8a5a1454',
 '124a15d998dfe011681819d7c792f7bb',
 '125e5760a3ff01937be3b2a1a7f14aa6',
 '1427b126f61597524866770b05d4eed2',
 '143c616ec3fe0cb065c3805cf9bd812e',
 '145140d23f166a1166dc4437107df8cc']

In [ ]:
## Getting Product names froom prediction 

predictions = pd.DataFrame(Recommend[:20])
predictions.columns = ['Product_ID']
predictions

,Product_ID
0,01422266d7a3131403364787ef9dab11
1,01ff1ff8aa5dec93e9938b989393a4ca
2,045e41f597e72ebfe5e9de50788fbf03
3,0573b564a90d2d9f9c460eb963da8c33
4,058b372f969b59e8c4a27e224243909c
5,05c11568c221bcac5790a8720019d141
6,06dce482ace017e78349b6f1fa83a022
7,07ffc018eaf23e086370dea42c74077b
8,095fc85516c1210e6dbed95ed8041ee0
9,09c3d39641970009c198caed304ccfc4


In [ ]:
predictions['Product_ID'] = predictions.Product_ID.apply(lambda x : Olist_Reg_db_[Olist_Reg_db_.PRODUCT_ID == x]['PRODUCT_ID'].unique()[0])

In [ ]:
predictions[:]

,Product_ID
0,01422266d7a3131403364787ef9dab11
1,01ff1ff8aa5dec93e9938b989393a4ca
2,045e41f597e72ebfe5e9de50788fbf03
3,0573b564a90d2d9f9c460eb963da8c33
4,058b372f969b59e8c4a27e224243909c
5,05c11568c221bcac5790a8720019d141
6,06dce482ace017e78349b6f1fa83a022
7,07ffc018eaf23e086370dea42c74077b
8,095fc85516c1210e6dbed95ed8041ee0
9,09c3d39641970009c198caed304ccfc4


In [ ]:
Olist_Reg_db_.to_csv('Olist_Reg_db_ALS.csv', index=False)

In [ ]:
X.to_csv('X_ALS_Reg.csv')

In [ ]:
correlation_matrix

array([[ 1.        ,  0.3375158 ,  0.13872829, ...,  0.74705912,
         0.32030689,  0.24635814],
       [ 0.3375158 ,  1.        , -0.47449314, ..., -0.29869214,
         0.52021476, -0.49346152],
       [ 0.13872829, -0.47449314,  1.        , ...,  0.47674136,
        -0.20576848,  0.18219044],
       ...,
       [ 0.74705912, -0.29869214,  0.47674136, ...,  1.        ,
        -0.02487883,  0.35444819],
       [ 0.32030689,  0.52021476, -0.20576848, ..., -0.02487883,
         1.        , -0.52338223],
       [ 0.24635814, -0.49346152,  0.18219044, ...,  0.35444819,
        -0.52338223,  1.        ]])

In [ ]:
correlation_matrix.shape

(1193, 1193)

In [ ]:
import json

with open('correlation_matrix_ALS_Reg.txt', 'w') as filehandle:
    json.dump(correlation_matrix.tolist(), filehandle)

In [ ]:
Olist_Reg_db_ = pd.read_csv('Olist_Reg_db_ALS.csv')

In [ ]:
X = pd.read_csv('X_ALS_Reg.csv', index_col=0)
X

,006431d77c665533ac99b501d8a6ab06,007da0ded5420c2ac9cdcdc758841cde,0096dd625383caa8afe2c028018bf4ce,00c2781c163b9f057924ed8c45c724d4,01414201d70bd508e1eee33ff67d2453,01503d0b64036165c2f202b538de9308,0195699b3388a65b55eab3dd04f48d1a,01cc8550598fb6140923843e6c06326e,0222b08d0816bba0acbcc31ba1d6cc1a,022bdd440f8a37e8ba62d67287135a75,023fafd700dff9fcc64d28e2defe98b9,02af4338e00cfed3af1a15854865879c,03346340b055f4770567d2a28ca52d4e,039a50ec3dc563eaf0bb3ecf176cdf99,03e3a430d50d6e1123627d133c65d5c4,03e454af33e93cda33e1b22175ab8361,03e5e57dd377a3da84cec48a7bbfba2a,03f2dda26a1b89b7cf91dcd8a41feb96,040a6b33a803ef269d0bf0c285cb2390,042b53f9e609289d8f6fcbd59b4acf63,04612c0745e93a575dac8d71974c805d,0499e8d702425ef8b10da2172f9fec96,053a441e831f4858b13182fe2671e967,05486eac340ae6a8f4837c6ab027d5ed,054c3d6634ff610c71c882d6f157bb34,0641f390b82e408dd226cd5e98327258,06d99479baac3413561b21bd7ac27767,06deea7d56068efd37c65f8daff0e03b,078d1ab1e800b24415a6d90e582df853,08029b648bf207edf7c24f3673382bf2,0879c47e542184edf3c921d2fee17084,08a0a61f98f22952682f7e463ef2d43e,08e7487578bd3fb98d11c5d95f1e5db5,0900a5f39daf5f37e39f42894f36a85d,0980d72e94bede8b0f7d8a4cd32511c8,0987ae71e7ad93d01d1202b7299fbb50,098ff9996de8b54d861bf8da445ba2ca,09ca1779c44a99be8f22fb0acfd3eefd,09cc193befa14d698869667e92148ab2,09e01f974b9533109a46281624e39808,...,f3ad475cc60e554f7113f18ab06810ca,f40e11fcab48bea5f847eff467a841a0,f40fb1a58c563f8e1581d3e93ce79507,f43d9a5994fb99bb2e14160e12666442,f4608a440b868f3f57f378c4b8cd643f,f4984d9f9ce9b3201db4932ea329cb40,f4c13379ddd0ed4f4fc1c0b49c2c5e51,f4e7ab7598cbef510a638a6068fbc56a,f59d1a0dc2219d4bdb4fc23f0d1d3c07,f665e06d0376b22d2ac11e29e369d6b1,f6a6b6537e91cdb94efd469d48d78690,f6a9bfc46722d91dc2306b08a0b8e11a,f6abfe23cd8425cc79033317e567b969,f74f7d4e2664b99e1cd78ef0f906f4cb,f7c7eee947743e900cad930670b94341,f7fd335725f778c5e85be886b3ac21ee,f85444670597b08b2c8a44beda3d72db,f8a069c2a2ce5ac98d668ad6a7950cb8,f8d4fd17a8f45986298a522b397522d0,f99b5ad76ac1085ae3628a3dc0eff0af,f9b4364d72fa2633b5eaaf6626e19cd9,f9efa1e08ec36a3bd42cdeb68995af0b,fa8047426049005aaa90db11b5fd39ad,faa2debf21e388f071f302ef5dc5e1cc,fab4d0e05c214c4baa684e6107d77757,fac3296ec4f116faccfa6a6d7e18b42f,fac60d7c4df3896a5af5db6c91e03797,fb54e4c6db413239a97873a13c366310,fbd2a08da7b03d6aa249b3b68ce13170,fbdea18ec346682a0dd7c542c56e74ff,fcfa6b64765235e944c28d83f99ee9dd,fd03d461d47cf24785f5e7ee7a1c9708,fd2ae8c714a773f85ef1b53fbebeda1f,fd2bc7895fa1c769be64f398f45c8dcf,fd44d124e2a3efe1398e5b4919b17413,fd92e79c1dfcd1ddae4d4630fef5bb7e,fec7723925e22866dab2debf6a38b695,fed5b7561b589c773a4a7d7028d0ae9b,ff349ea42c06633992fc3c94c3da0f58,ffc436ef9df6ede268ee1548a0dfbbba
PRODUCT_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
00ae7076313576f94d9107599d79a978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00faa46f36261af8bbf3a4d37fa4841b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0105b5323d24fc655f73052694dbbb3a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01365901d8f2247976a7e5d4c98c52c0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [ ]:
import json
with open('correlation_matrix_ALS_Reg.txt') as f:
    correlation_matrix = json.load(f)

In [ ]:
correlation_matrix = np.array(correlation_matrix)

In [ ]:
#Olist_db_ = pd.read_csv('Olist_db_ALS.csv')
#X = pd.read_csv('X_ALS.csv')
    
#with open('correlation_matrix_ALS.txt') as f:
#    correlation_matrix = json.load(f)

#correlation_matrix = np.array(correlation_matrix)

def Recommendations_ALS(Customer_id):
    #Olist_db_ = Olist_db_[Olist_db_.Customer_ID < 3600]
    fave_prod = Olist_Reg_db_.groupby(['CUSTOMER_ID']).max()['PRODUCT_ID'].to_frame()
    fave_prod = fave_prod.reset_index()
    
    #prd_id = get_prod_id(Customer_id)
    prd_id = fave_prod[fave_prod.CUSTOMER_ID == Customer_id]['PRODUCT_ID']
    Product_id = prd_id.iloc[0]
    product_names = list(X.index)
    product_ID_Idx = product_names.index(Product_id)
    
    correlation_product_ID = correlation_matrix[product_ID_Idx]
    
    Recommend = list(X.index[correlation_product_ID > 0.70])
    # Removes the item already bought by the customer
    Recommend.remove(Product_id) 
    
    ## Getting Product names froom prediction 

    predictions = pd.DataFrame(Recommend[:20])
    predictions.columns = ['Product_ID']

    predictions['Product_ID'] = predictions.Product_ID.apply(lambda x : Olist_Reg_db_[Olist_Reg_db_.PRODUCT_ID == x]['PRODUCT_ID'].unique()[0])
    Recommendations = predictions[:10]
    return Recommendations

In [ ]:
Recommendations = Recommendations_ALS('fed5b7561b589c773a4a7d7028d0ae9b')
Recommendations[:10]

,Product_ID
0,01422266d7a3131403364787ef9dab11
1,01ff1ff8aa5dec93e9938b989393a4ca
2,045e41f597e72ebfe5e9de50788fbf03
3,0573b564a90d2d9f9c460eb963da8c33
4,058b372f969b59e8c4a27e224243909c
5,05c11568c221bcac5790a8720019d141
6,06dce482ace017e78349b6f1fa83a022
7,07ffc018eaf23e086370dea42c74077b
8,095fc85516c1210e6dbed95ed8041ee0
9,09c3d39641970009c198caed304ccfc4


In [ ]:
from google.colab import files
files.download("X_ALS_Reg.csv")